In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch09DataSources").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_16455


In [5]:
csvFile = spark.read.format("csv")\
  .option("header", "true")\
  .option("mode", "FAILFAST")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/dataSets/spark-guide/flight-data/csv/2010-summary.csv")

In [6]:
csvFile.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [10]:
csvFile.count()

255

In [9]:
# COMMAND ----------

csvFile.write.format("csv").mode("append").option("sep", "\t")\
  .save("/user/kranthidr/temp/my-tsv-file.tsv")

In [12]:
# csvFile1 = spark.read.format("csv")\
#   .option("header", "true")\
#   .option("mode", "FAILFAST")\
#   .option("inferSchema", "true")\
#   .load("/user/kranthidr/temp/my-tsv-file.tsv")
# csvFile1.count()

# Py4JJavaError: An error occurred while calling o107.count.
# : org.apache.spark.SparkException: Job aborted due to stage failure: 
#             Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 
#                 (TID 17, wn05.itversity.com, executor 7): 
#         org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST.

In [13]:
csvFile1 = spark.read.format("csv")\
  .option("header", "true")\
  .option("mode", "dropMalformed")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/temp/my-tsv-file.tsv")
csvFile1.show(5)

+-----------------------+
|United States	Romania	1|
+-----------------------+
|   United States	Ire...|
|   United States	Ind...|
|   Egypt	United Stat...|
|   Equatorial Guinea...|
|   United States	Sin...|
+-----------------------+
only showing top 5 rows



In [14]:
csvFile1 = spark.read.format("csv")\
  .option("header", "true")\
  .option("mode", "permissive")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/temp/my-tsv-file.tsv")
csvFile1.show(5)

+-----------------------+
|United States	Romania	1|
+-----------------------+
|   United States	Ire...|
|   United States	Ind...|
|   Egypt	United Stat...|
|   Equatorial Guinea...|
|   United States	Sin...|
+-----------------------+
only showing top 5 rows



In [16]:
csvFile1 = spark.read.format("com.databricks.spark.csv")\
    .option("delimiter", "\t")\
  .option("header", "true")\
  .option("mode", "permissive")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/temp/my-tsv-file.tsv")
csvFile1.count()

508

In [17]:
# COMMAND ----------
spark.read.format("json").option("mode", "FAILFAST")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/dataSets/spark-guide/flight-data/json/2010-summary.json").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [18]:
# COMMAND ----------

csvFile.write.format("json").mode("overwrite").save("/user/kranthidr/temp/my-json-file.json")

In [19]:
# COMMAND ----------

spark.read.format("parquet")\
  .load("/user/kranthidr/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [20]:
# COMMAND ----------

csvFile.write.format("parquet").mode("overwrite")\
  .save("/user/kranthidr/temp/my-parquet-file.parquet")

In [21]:
# COMMAND ----------

spark.read.format("orc").load("/user/kranthidr/dataSets/spark-guide/flight-data/orc/2010-summary.orc").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [22]:
# COMMAND ----------

csvFile.write.format("orc").mode("overwrite").save("/user/kranthidr/temp/my-json-file.orc")